# YOLOv5 fine-tuning for mask detection

In [ ]:
import os
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from glob import glob

In [ ]:
images_path = 'data/images'
annotations_path = 'data/annotations'
yolo_annotations_path = 'data/'
train_images_path = os.path.join(yolo_annotations_path, 'train/images')
train_labels_path = os.path.join(yolo_annotations_path, 'train/labels')
val_images_path = os.path.join(yolo_annotations_path, 'val/images')
val_labels_path = os.path.join(yolo_annotations_path, 'val/labels')

# Create output directories
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

In [ ]:
class_mapping = {'without_mask': 0, 'with_mask': 1}

In [ ]:
def convert_to_yolo(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (box[0] + box[1]) / 2.0 - 1
    y_center = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    return [x_center * dw, y_center * dh, w * dw, h * dh]

In [ ]:
xml_files = glob(os.path.join(annotations_path, '*.xml'))
image_files = glob(os.path.join(images_path, '*.png'))
image_files = sorted(image_files)

In [ ]:
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

In [ ]:
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    root = tree.getroot()
    image_filename = root.find('filename').text
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    img_path = os.path.join(images_path, image_filename)
    
    yolo_filename = os.path.splitext(image_filename)[0] + '.txt'
    yolo_data = []
    
    for obj in root.iter('object'):
        class_name = obj.find('name').text
        if class_name not in class_mapping:
            continue
        class_id = class_mapping[class_name]
        
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
        bbox = convert_to_yolo((width, height), (xmin, xmax, ymin, ymax))
        yolo_data.append(f"{class_id} {' '.join(map(str, bbox))}")
    
    if img_path in train_files:
        # Write YOLO annotation
        with open(os.path.join(train_labels_path, yolo_filename), 'w') as f:
            f.write('\n'.join(yolo_data))

        os.system(f"cp '{img_path}' '{train_images_path}'")
    else:
        # Write YOLO annotation
        with open(os.path.join(val_labels_path, yolo_filename), 'w') as f:
            f.write('\n'.join(yolo_data))
        # Copy image
        os.system(f"cp '{img_path}' '{val_images_path}'")

print("Conversion to YOLO format and dataset split completed.")

Conversion to YOLO format and dataset split completed.
